# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

## Лабораторная работа 13

__При решении данных задач не подразумевается использования других коллекций, кроме `dask.DataFrame`, если в задании явно не указано обратное.__

In [69]:
import dask.dataframe as dd
import pandas as pd
import dask.bag as db
import json
import dask

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

In [70]:
reсipes_full = dd.read_csv('recipes_full_*.csv', parse_dates = ['submitted'], assume_missing = True)
reсipes_full.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970.0,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9.0
1,1089012.0,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5.0
2,1428572.0,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5.0
3,1400250.0,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1.0
4,387709.0,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10.0


In [71]:
# assume_missing может указать Dask на необходимость исползования float для столбцов содержащих значения int, 
# что позволит хранить в этих столбцах значения np.NaN для пропущенных значений

2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

In [72]:
reсipes_full.npartitions

16

In [73]:
reсipes_full.dtypes

id                       float64
name                      object
minutes                  float64
contributor_id           float64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

In [74]:
reсipes_full.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970.0,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9.0
1,1089012.0,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5.0
2,1428572.0,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5.0
3,1400250.0,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1.0
4,387709.0,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10.0


In [75]:
reсipes_full.tail()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
44011,1029131.0,tuti waffle snackies steakhouse,19.0,171345.0,1973-10-18,4.0,"according to a providence journal article, ama...",4.0
44012,1700703.0,noelias cheats crocante fleisch zitumbuwa,1.0,30228.0,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1.0
44013,1910650.0,rubbed restuffed pelmeni bedouin flavourful,60.0,591905.0,2009-09-26,3.0,another great recipe to add to the growing swe...,2.0
44014,713836.0,stems polpettine peezi,NaN,357389.0,2003-09-30,4.0,adapted from top secret recipes. love this!,9.0
44015,660699.0,clementines,64.0,29196.0,1973-06-03,6.0,this would make a great start to your holiday ...,8.0


4. Посчитайте, сколько строк содержит каждый из блоков (partitions).

In [76]:
reсipes_full.map_partitions(len).compute()

0     237505
1      41450
2     235222
3      43733
4     238139
5      40816
6     234750
7      44205
8     237887
9      41068
10    234486
11     44468
12    238298
13     40656
14    234938
15     44016
dtype: int64

5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` при решении данной задачи.

In [77]:
reсipes_full['n_steps'].max().compute()

145.0

In [78]:
#reсipes_full['n_steps'].map_partitions(max).compute()

In [79]:
# mac ne lubit vizualizirovat'

6. Посчитайте количество рецептов с группировкой по месяцам добавления отзыва в базу.

In [80]:
rec_month_count = reсipes_full.groupby(reсipes_full['submitted'].dt.month).id.count().compute()
rec_month_count

submitted
1     193363
2     173834
3     192389
4     186049
5     192487
6     184205
7     189337
8     187276
9     181081
10    187018
11    180974
12    183624
Name: id, dtype: int64

7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

In [81]:
import re

def add_rating(data):
    rev = json.loads(data[0])
    rev['rating'] = int(re.findall(r'reviews_(\d).json', data[1])[0])
    return rev
    
reviews_ = db.read_text('reviews_*.json', include_path = True).map(add_rating)
reviews_.take(1)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},)

In [82]:
reviews_full = reviews_.to_dataframe()
reviews_full.head()

,user_id,recipe_id,date,review,rating
0,452355,292657,2016-05-08,WOW!!! This is the best. I have never been abl...,0
1,329304,433404,2006-06-14,This was good but the dressing needed somethin...,0
2,227932,2008187,1985-11-19,"Very good,it was a hit for my family. I used 6...",0
3,171468,270716,2019-05-21,Made for ZWT-8 Family Picks after I saw these ...,0
4,91392,1159916,1972-09-18,Very nice slaw. I especially like that it does...,0


In [83]:
reviews_full.dtypes

user_id       int64
recipe_id     int64
date         object
review       object
rating        int64
dtype: object

In [84]:
reviews_full['date'] = dd.to_datetime(reviews_full.date)

In [85]:
reviews_full.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
review               object
rating                int64
dtype: object

In [86]:
reviews_full.head()

,user_id,recipe_id,date,review,rating
0,452355,292657,2016-05-08,WOW!!! This is the best. I have never been abl...,0
1,329304,433404,2006-06-14,This was good but the dressing needed somethin...,0
2,227932,2008187,1985-11-19,"Very good,it was a hit for my family. I used 6...",0
3,171468,270716,2019-05-21,Made for ZWT-8 Family Picks after I saw these ...,0
4,91392,1159916,1972-09-18,Very nice slaw. I especially like that it does...,0


In [87]:
rev_month_mean = reviews_full.groupby(reviews_full['date'].dt.month).rating.mean().compute()
rev_month_mean

date
1     4.408553
2     4.407135
3     4.406911
4     4.412259
5     4.415401
6     4.418419
7     4.411717
8     4.410822
9     4.410661
10    4.411452
11    4.408829
12    4.406908
Name: rating, dtype: float64

8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`

In [90]:
df8 = pd.DataFrame(dict(rev_month_mean = rev_month_mean, rec_month_count = rec_month_count))
df8.rename(columns = {'rev_month_mean' : 'mean_rating', 'rec_month_count' : 'recipes_count'})
df8

,rev_month_mean,rec_month_count
1,4.408553,193363
2,4.407135,173834
3,4.406911,192389
4,4.412259,186049
5,4.415401,192487
6,4.418419,184205
7,4.411717,189337
8,4.410822,187276
9,4.410661,181081
10,4.411452,187018
